# 7장. 앙상블 학습과 랜덤 포레스트

https://nbviewer.jupyter.org/github/rickiepark/handson-ml/tree/master/

이 코드의 내용은 Hands-On Machine Learning with Scikit-Learn & TensorFlow을 참고했음을 밝힙니다.

* 앙상블 : 일련의 예측기
* 앙상블 학습(Ensemble Learning) : 일련의 예측기(분류나 회귀 모델)로부터 예측을 수집하면 가장 좋은 모델 하나보다 더 좋은 예측을 얻을 수 있을 것입니다. 예를 들어 훈련 세트로부터 무작위로 각기 다른 서브셋을 만들어 일련의 결정 트리 분류기를 훈련시킬 수 있습니다. 예측을 하려면 모든 개별 트리의 예측을 구하면 됩니다. 그런 다음 가장 많은 선택을 받은 클래스를 예측으로 삼습니다.
* 앙상블 방법(Ensemble method) : 앙상블 학습 알고리즘
* 랜덤 포레스트(Random Forest) : 결정 트리의 앙상블, 오늘날 가장 강력한 머신러닝 알고리즘 중 하나

##  투표 기반 분류기(p242)

![image.png](https://i.imgur.com/KAQggSN.png)

* 직접 투표(hard voting) 분류기 : 다수결 투표로 정해지는 분류기(각 분류기의 예측을 모아서 가장 많이 선택된 클래스를 예측하는 것)
* 조금 놀랍게도 이 다수결 투표 분류기가 앙상블에 포함된 개별 분류기 중 가장 뛰어난 것보다도 정확도가 높을 경우가 많습니다. 사실 각 분류기가 약한 학습기(weak learner, 즉 랜덤 추측보다 조금 더 높은 성능을 내는 분류기)일지라도 충분하게 많고 다양하다면 앙상블은 (높은 정확도를 내는) 강한 학습기(strong learner)가 될 수 있습니다.
* 앙상블 방법은 예측기가 가능한 한 서로 독립적일 때 최고의 성능을 발휘합니다. 다양한 분류기를 얻는 한 가지 방법은 각기 다른 알고리즘으로 학습시키는 것입니다. 이렇게 하면 매우 다른 종류의 오차를 만들 가능성이 높기 때문에 앙상블 모델의 정확도를 향상시킵니다.

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression(solver='liblinear')
rnd_clf = RandomForestClassifier(n_estimators=10)
svm_clf = SVC(gamma = 'auto')

voting_clf = VotingClassifier(
estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
voting='hard')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='warn',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='liblinear', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     class_weight=None,
                                                     criterion='g...
                                        

In [13]:
# 각 분류기의 테스트셋 정확도를 확인
# 투표 기반 분류기가 다른 개별 분류기보다 성능이 조금 더 높다.

from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.864
RandomForestClassifier 0.872
SVC 0.888
VotingClassifier 0.896


모든 분류기가 클래스의 확률을 예측할 수 있으면(즉, predict_proba() 메서드가 있으면), 개별 분류기의 예측을 평균 내어 확률이 가장 높은 클래스를 예측할 수 있습니다. 이를 간접 투표(soft voting)라고 합니다. 이 방식은 확률이 높은 투표에 비중을 더 두기 때문에 직접 투표 방식보다 성능이 높습니다. 이 방식을 사용하기 위해서는 voting='hard'를 voting='soft'로 바꾸고 모든 분류기가 클래스의 확률을 추정할 수 있으면 됩니다. SVC는 기본값에서는 클래스 확률을 제공하지 않으므로 probability 매개변수를 True로 지정해야 합니다(이렇게 하면 클래스 확률을 추정하기 위해 교차 검증을 사용하므로 훈련 속도가 느려지지만 SVC에서 predict_proba() 메서드를 사용할 수 있습니다). 앞의 코드를 간접 투표 방식을 사용하도록 변경하면 91%의 정확도를 얻을 것입니다.

## 배깅과 페이스팅(p245)

앞서 말했듯이 다양한 분류기를 만드는 한 가지 방법은 각기 다른 훈련 알고리즘을 사용하는 것입니다. 또 다른 방법은 같은 알고리즘을 사용하지만 훈련 세트의 서브셋을 무작위로 구성하여 분류기를 각기 다르게 학습시키는 것입니다. 훈련 세트에서 중복을 허용하여 샘플링하는 방식을 배깅(bagging, bootstrap aggregating)이라 하며, 중복을 허용하지 않고 샘플링하는 방식을 페이스팅(pasting)이라고 합니다.<br>
다시 말해 배깅과 페이스팅에서는 같은 훈련 샘플을 여러 개의 예측기에 걸쳐 사용할 수 있습니다. 하지만 배깅만이 한 예측기를 위해 같은 훈련 샘플을 여러 번 샘플링할 수 있습니다.

![image.png](https://i.imgur.com/ULRSweQ.png)

모든 예측기가 훈련을 마치면 앙상블은 모든 예측기의 예측을 모아서 새로운 샘플에 대한 예측을 만듭니다. 수집 함수는 전형적으로 분류일 때는 통계적 최빈값(statistical mode, 즉 직접 투표 분류기처럼 가장 많은 예측 결과)이고 회귀에 대해서는 평균을 계산합니다. 개별 예측기는 원본 훈련 세트로 훈련시킨 것보다 훨씬 크게 편향되어 있지만 수집 함수를 통과하면 편향과 분산이 모두 감소합니다. 일반적으로 앙상블의 결과는 원본 데이터셋으로 하나의 예측기를 훈련시킬때와 비교해 편향은 비슷하지만 분산은 줄어듭니다.<br>
위 그림에서 볼 수 있듯이 예측기는 모두 동시에 다른 CPU 코어나 서버에서 병렬로 학습시킬 수 있습니다. 이와 유사하게 예측도 병렬로 수행할 수 있습니다. 이런 확장성 덕분에 배깅과 페이스팅의 인기가 높습니다.

## 사이킷런의 배깅과 페이스팅(p246)

In [14]:
# 결정 트리 분류기 500개의 앙상블을 훈련시키는 코드입니다.
# 각 분류기는 훈련 세트에서 중복을 허용하여 무작위로 선택된 100개의 샘플로 훈련됩니다.
# 이는 배깅의 경우이고, 대신 페이스팅을 사용하려면 bootstrap=False로 지정하면 됩니다.
# n_jobs 매개변수는 사이킷런이 훈련과 예측에 사용할 CPU 코어 수를 지정합니다(-1로 지정하면 가용한 모든 코어를 사용합니다).

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
DecisionTreeClassifier(), n_estimators=500,
max_samples=100, bootstrap=True, n_jobs=-1)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

BaggingClassifier는 기반이 되는 분류기가 결정 트리 분류기처럼 클래스 확률을 추정할 수 있으면(즉, predict_proba() 함수가 있으면) 직접 투표 대신 자동으로 간접 투표 방식을 사용합니다.

![image.png](https://i.imgur.com/VYSpCt7.png)

위의 그림은 단일 결정 트리의 결정 경계와 500개의 트리를 사용한 배깅 앙상블의 결정 경계를 비교한 것입니다. 여기서 볼 수 있듯이 앙상블의 예측이 결정 트리 하나의 예측보다 일반화가 훨씬 잘된 것 같습니다. 앙상블은 비슷한 편향에서 더 작은 분산을 만듭니다(훈련 세트의 오차 수가 거의 비슷하지만 결정 경계는 덜 불규칙합니다).

![image.png](https://i.imgur.com/HheiIaj.png)

부트스트래핑은 각 예측기가 학습하는 서브셋에 다양성을 증가시키므로 배깅이 페이스팅보다 편향이 조금 더 높습니다. 하지만 이는 예측기들의 상관관계를 줄이므로 앙상블의 분산을 감소시킵니다. 전반적으로 배깅이 더 나은 모델을 만들기 때문에 일반적으로 더 선호합니다. 그러나 시간과 CPU 파워에 여유가 있다면 교차 검증으로 배깅과 페이스팅을 모두 평가해서 더 나은 쪽을 선택하는 것이 좋습니다.

## oob 평가

배깅을 사용하면 어떤 샘플은 한 예측기를 위해 여러 번 샘플링되고 어떤 것은 전혀 선택되지 않을 수 있습니다. BaggingClassifier는 기본값으로 중복을 허용하여(bootstrap=True) 훈련 세트의 크기 만큼인 m개 샘플을 선택합니다. 이는 평균적으로 각 예측기의 훈련 샘플의 63% 정도만 샘플링된다는 것을 의미합니다. 선택되지 않은 훈련 샘플의 나머지 37%를 oob(out-of-bag)샘플이라고 부릅니다. 예측기마다 남겨진 37%는 모두 다릅니다.<br>
예측기가 훈련되는 동안에는 oob 샘플을 사용하지 않으므로 검증 세트나 교차 검정을 사용하지 않고 oob 샘플을 사용해 평가할 수 있습니다. 앙상블의 평가는 각 예측기의 oob 평가를 평균하여 얻습니다.

In [5]:
# Pr(i in sample) = 1 - Pr(i not in sample)

1 - 0.99**100

0.6339676587267709

In [21]:
# 사이킷런에서 BaggingClassifier를 만들 때 oob_score=True로 지정하면 훈련이 끝난 후 자동으로 oob 평가를 수행합니다.
# 평가 점수 결과는 oob_score_ 변수에 저장되어 있습니다.
# oob 평가 결과를 보면 이 BaggingClassifier는 테스트 세트에서 약 90.1%의 정확도를 얻을 것으로 보입니다.

bag_clf = BaggingClassifier(
DecisionTreeClassifier(), n_estimators=500,
bootstrap=True, n_jobs=-1, oob_score=True)

bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.9013333333333333

In [22]:
# oob 평가 결과와 테스트 세트에서의 정확도가 비슷하다는 것을 확인할 수 있습니다.

from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.912

In [24]:
# oob 샘플에 대한 결정 함수의 값도 oob_decision_function_ 변수에서 확인할 수 있습니다.
# 이 경우 결정 함수는 각 훈련 샘플의 클래스 확률을 반환합니다(기반이 되는 예측기가 predict_proba() 메서드를 가지고 있기 때문에).
# 다음 예를 보면 oob 평가는 첫 번째 훈련 샘플이 양성 클래스에 속할 확률을 61.5%로 추정하고 있습니다(그리고 음성 클래스에 속할 확률은 38.5%입니다).

bag_clf.oob_decision_function_

array([[0.385     , 0.615     ],
       [0.36263736, 0.63736264],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.05714286, 0.94285714],
       [0.33333333, 0.66666667],
       [0.02150538, 0.97849462],
       [0.98918919, 0.01081081],
       [0.98684211, 0.01315789],
       [0.78212291, 0.21787709],
       [0.        , 1.        ],
       [0.83928571, 0.16071429],
       [0.82198953, 0.17801047],
       [0.97222222, 0.02777778],
       [0.06818182, 0.93181818],
       [0.        , 1.        ],
       [0.98843931, 0.01156069],
       [0.96511628, 0.03488372],
       [0.99465241, 0.00534759],
       [0.02617801, 0.97382199],
       [0.32124352, 0.67875648],
       [0.86538462, 0.13461538],
       [1.        , 0.        ],
       [0.96511628, 0.03488372],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.61748634, 0.38251366],
       [0.

## 랜덤 패치와 랜덤 서브스페이스(p249)

BaggingClassifier는 특성 샘플링도 지원합니다. max_features, bootstrap_features 두 매개변수로 조절됩니다. 작동 방식은 max_samples, bootstrap과 동일하지만 샘플이 아니고 특성에 대한 샘플링입니다. 따라서 각 예측기는 무작위로 선택한 입력 특성의 일부분으로 훈련됩니다.<br>
특히 (이미지와 같은) 매우 고차원의 데이터셋을 다룰 때 유용합니다. 훈련 특성과 샘플을 모두 샘플링하는 것을 랜덤 패치 방식(Random Patches method)이라고 합니다. 훈련 샘플을 모두 사용하고(즉, bootstrap=False이고 max_samples=1.0) 특성은 샘플링하는(즉, bootstrap_features=True이고 max_features는 1.0보다 작은) 것을 랜덤 서브스페이스 방식(Random Subspaces method)이라고 합니다.<br>
특성 샘플링은 더 다양한 예측기를 만들며 편향을 늘리는 대신 분산을 낮춥니다.

## 랜덤 포레스트(p250)

앞서 언급했듯이 랜덤 포레스트는 일반적으로 배깅 방법(또는 페이스팅)을 적용한 결정 트리의 앙상블입니다. 전형적으로 max_samples를 훈련 세트의 크기로 지정합니다. BaggingClassifier에 DecisionTreeClassifier를 넣어 만드는 대신 결정 트리에 최적화되어 사용하기 편리한 RandomForestClassifier를 사용할 수 있습니다(비슷하게 회귀 문제를 위한 RandomForestRegressor가 있습니다).

<img src='https://i.imgur.com/ikbgIUZ.png' width='100%'>

### Note
max_samples를 지정하지 않으면 데이터의 수만큼 샘플을 추출한다.

In [25]:
# 다음은 (최대 16개의 리프 노드를 갖는) 500개 트리로 이뤄진 랜덤 포레스트 분류기를 여러 CPU 코어에서 훈련시키는 코드입니다.

from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

* RandomForestClassifier는 몇 가지 예외가 있지만 (트리 성장의 조절을 위한) DecisionTreeClassifier의 매개변수와 앙상블 자체를 제어하는 데 필요한 BaggingClassifier의 매개변수를 모두 가지고 있습니다.
* 랜덤 포레스트 알고리즘은 트리의 노드를 분할할 때 전체 특성 중에서 최선의 특성을 찾는 대신 무작위로 선택한 특성 후보 중에서 최적의 특성을 찾는 식으로 무작위성을 더 주입합니다. 이는 결국 트리를 더욱 다양하게 만들고 (다시 한번) 편향을 손해 보는 대신 분산을 낮추어 전체적으로 더 훌륭한 모델을 만들어냅니다.

In [26]:
# 다음은 BaggingClassifier를 사용해 앞의 RandomForestClassifier와 거의 유사하게 만든 것입니다.

bag_clf = BaggingClassifier(
DecisionTreeClassifier(splitter='random', max_leaf_nodes=16),
n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1)

## 엑스트라 트리(p251)

랜덤 포레스트에서 트리를 만들 때 (앞서 이야기한 것처럼) 각 노드는 무작위로 특성의 서브셋을 만들어 분할에 사용합니다. 트리를 더욱 무작위하게 만들기 위해 (보통의 결정 트리처럼) 최적의 임계값을 찾는 대신 후보 특성을 사용해 무작위로 분할한 다음 그중에서 최상의 분할을 선택합니다.<br>
이와 같이 극단적으로 무작위한 트리의 랜덤 포레스트를 익스트림 랜덤 트리(Extremely Randomized Trees)앙상블(또는 줄여서 엑스트라 트리(Extra-Tree))이라고 부릅니다. 여기서도 역시 편향이 늘어나지만 대신 분산을 낮추게 됩니다. 모든 노드에서 특성마다 가장 최적의 임곗값을 찾는 것이 트리 알고리즘에서 가장 시간이 많이 소요되는 작업 중 하나이므로 일반적인 랜덤 포레스트보다 엑스트라 트리가 훨씬 빠릅니다.<br>
엑스트라 트리를 만들려면 사이킷런의 ExtraTreesClassifier를 사용합니다. 사용법은 RandomForestClassifier와 같습니다. 마찬가지로 ExtraTreesRegressor도 RandomForestRegressor와 같은 API를 제공합니다.<br>
RandomForestClassifier가 ExtraTreeClassifier보다 더 나을지 혹은 나쁠지 예단하긴 어렵습니다. 일반적으로 둘 다 시도해보고 교차 검증으로 비교해보는 것이 유일한 방법입니다(그리고 그리드 탐색으로 하이퍼파라미터 튜닝을 합니다).

## 특성 중요도

랜덤 포레스트의 또 다른 장점은 특성의 상대적 중요도를 측정하기 쉽다는 것입니다. 사이킷런은 어떤 특성을 사용한 노드가 (랜덤 포레스트에 있는 모든 트리에 걸쳐서) 평균적으로 불순도를 얼마나 감소시키는지 확인하여 특성의 중요도를 측정합니다. 더 정확히 말하면 가중치 평균이며 각 노드의 가중치는 연관된 훈련 샘플 수와 같습니다.<br>
결정 트리의 특성 중요도는 노드에 사용된 특성별로 (현재 노드의 샘플 비율 \* 불순도) - (왼쪽 자식 노드의 샘플 비율 \* 불순도) - (오른쪽 자식 노드의 샘플 비율 \* 불순도)와 같이 계산하여 더하고, 특성 중요도의 합이 1이 되도록 전체 합으로 나누어 정규화합니다. 여기서 샘플 비율은 트리 전체 샘플 수에 대한 비율입니다. 랜덤 포레스트의 특성 중요도는 각 결정 트리의 특성 중요도를 모두 계산하여 더한 후 트리 수로 나눈 것입니다.<br>
사이킷런은 훈련이 끝난 뒤 특성마다 자동으로 이 점수를 계산하고 중요도의 전체 합이 1이 되도록 결괏값을 정규화합니다. 이 값은 feature_importances_ 변수에 저장되어 있습니다.

In [27]:
# 다음 코드는 iris 데이터셋에 RandomForestClassifier를 훈련시키고 각 특성의 중요도를 출력합니다.
# 가장 중요한 특성은 꽃잎의 길이(44%)와 너비(42%)이고 꽃받침의 길이와 너비는 비교적 덜 중요해 보입니다(각각 11%와 2%).

from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris['data'], iris['target'])
for name, score in zip(iris['feature_names'], rnd_clf.feature_importances_):
    print(name, score)

sepal length (cm) 0.09932445849018497
sepal width (cm) 0.024801230837113882
petal length (cm) 0.4476460757250039
petal width (cm) 0.42822823494769713


이와 유사하게 MNIST 데이터셋에 랜덤 포레스트 분류기를 훈련시키고 각 픽셀의 중요도를 그래프로 나타내면 아래 그림과 같은 이미지를 얻게 됩니다.<br>
![image.png](https://i.imgur.com/7wuft6j.png)<br>
랜덤 포레스트는 특히 특성을 선택해야 할 때 어떤 특성이 중요한지 빠르게 확인할 수 있어 매우 편리합니다.

## 부스팅(boosting)(p253)

* 원래는 가설 부스팅(hypothesis boosting)이라 불렸습니다.
* 약한 학습기를 여러 개 연결하여 강한 학습기를 만드는 앙상블 방법을 말합니다.
* 부스팅 방법의 아이디어는 앞의 모델을 보완해나가면서 일련의 예측기를 학습시키는 것입니다.
* 가장 인기 있는 것은 아다부스트(AdaBoost)와 그래디언트 부스팅(Gradient Boosting)입니다.

## 아다부스트(p253)

이전 예측기를 보완하는 새로운 예측기를 만드는 방법은 이전 모델이 과소적합했던 훈련 샘플의 가중치를 더 높이는 것입니다. 이렇게 하면 새로운 예측기는 학습하기 어려운 샘플에 점점 더 맞춰지게 됩니다. 이것이 아다부스트에서 사용하는 방식입니다.<br>
예를 들어 아다부스트 분류기를 만들려면 기반이 되는 첫 번째 분류기(예를 들면 결정 트리)를 훈련 세트에서 훈련시키고 에측을 만듭니다. 그다음에 잘못 분류된 훈련 샘플의 가중치를 상대적으로 높입니다. 두 번째 분류기는 업데이트된 가중치를 사용해 훈련 세트에서 훈련하고 다시 예측을 만듭니다. 그다음에 다시 가중체를 업데이트하는 식으로 계속됩니다.<br>
![image.png](https://i.imgur.com/rTWwY1g.png)

![image.png](https://i.imgur.com/DS1JzRS.png)<br>
위 그림은 moons 데이터셋에 훈련시킨 다섯 개의 연속된 예측기의 결정 경계입니다(이 모델은 규제를 강하게 한 RBF 커널 SVM 분류기입니다). 첫 번째 분류기가 많은 샘플을 잘못 분류해서 이 샘플들의 가중치가 높아졌습니다. 따라서 두 번째 분류기는 이 샘플들을 더 정확히 예측하게 됩니다. 오른쪽 그래프는 학습률을 반으로 낮춘 것만 빼고 똑같은 일련의 예측기를 나타낸 것입니다(즉, 잘못 분류된 샘플의 가중치는 반복마다 절반 정도만 높아집니다). 그림에서 볼 수 있듯이 이런 연속된 학습 기법은 경사 하강법과 비슷한 면이 있습니다. 경사 하강법은 비용 함수를 최소화하기 위해 한 예측기의 모델 파라미터를 조정해가는 반면 아다부스트는 점차 좋아지도록 앙상블에 예측기를 추가합니다.<br>
모든 예측기가 훈련을 마치면 이 앙상블은 배깅이나 페이스팅과 비슷한 방식으로 예측을 만듭니다. 하지만 가중치가 적용된 훈련 세트의 전반적인 정확도에 따라 예측기마다 다른 가중치가 적용됩니다.<br>
연속된 학습 기법에는 중요한 단점이 하나 있습니다. 각 예측기는 이전 예측기가 훈련되고 평가된 후에 학습될 수 있기 때문에 병렬화(또는 분할)를 할 수 없습니다. 결국 배깅이나 페이스팅만큼 확장성이 높지 않습니다.

아다부스트 알고리즘을 더 자세히 들여다봅시다. 각 샘플 가중치 w(i)는 초기에 1/m으로 초기화됩니다. 첫 번째 예측기가 학습되고, 가중치가 적용된 에러율 r1이 훈련 세트에 대해 계산됩니다.<br>
![image.png](https://i.imgur.com/avSV856.png)<br>
여기서 y_hat_j(i)는 i번째 샘플에 대한 j번째 예측기의 예측입니다.

예측기의 가중치 a_j는 아래 식을 사용해 계산됩니다. 여기서 n은 학습률 파라미터입니다(기본값 1, 원래 아다부스트 알고리즘은 학습률 하이퍼파라미터를 사용하지 않습니다). 예측기가 정확할수록 가중치가 더 높아지게 됩니다. 만약 무작위로 예측하는 정도라면 가중치가 0에 가까울 것입니다. 그러나 그보다 나쁘면(즉, 무작위 추측보다 정확도가 낮으면) 가중치는 음수가 됩니다. 무작위로 예측하여 에러율(r)이 0.5에 가까워지면 (1-r)/r = 1이 되므로 예측기 가중치가 0에 가까워집니다.<br>
에러율이 0.5보다 높으면 (1-r)/r < 1이 되어 예측기 가중치는 음수가 됩니다.
![image.png](https://i.imgur.com/qKCZN6H.png)

그 다음에는 아래 식을 사용해 샘플의 가중치를 업데이트합니다. 즉, 잘못 분류된 샘플의 가중치가 증가됩니다.<br>
![image.png](https://i.imgur.com/aCq46km.png)<br>
그런 다음 모든 샘플의 가중치를 정규화합니다(즉, 모든 w(i)들의 합으로 나눕니다).

마지막으로 새 예측기가 업데이트된 가중치를 사용해 훈련하고 전체 과정이 반복됩니다(새 예측기의 가중치가 계산되고 샘플의 가중치를 업데이트해서 또 다른 예측기를 훈련시키는 식입니다). 이 알고리즘은 지정된 예측기 수에 도달하거나 완벽한 예측기가 만들어지면 중단됩니다.<br>
예측을 할 때 아다부스트는 단순히 모든 예측기의 예측을 계산하고 예측기 가중치 a_j를 더해 예측 결과를 만듭니다. 가중치 합이 가장 큰 클래스가 예측 결과가 됩니다.<br>
![image.png](https://i.imgur.com/W1agF5T.png)

사이킷런은 SAMME라는 아다부스트의 다중 클래스 버전을 사용합니다. 클래스가 두 개 뿐일 때는 SAMME 아다부스트와 동일합니다. 예측기가 클래스의 확률을 추정할 수 있다면(즉, predict_proba() 메서드가 있다면) 사이킷런은 SAMME.R(끝의 R은 'Real'을 말합니다)이라는 변종을 사용합니다. 이 알고리즘은 예측값 대신 클래스 확률에 기반하며 일반적으로 성능이 더 좋습니다.

In [28]:
# 다음 코드는 사이킷런의 AdaBoostClassifier를 사용하여(AdaBoostRegressor도 존재) 200개의 아주 얕은 결정 트리를 기반으로 하는 아다부스트 분류기를 훈련시킵니다.
# 여기에서 사용하는 결정 트리는 max_depth = 1입니다.
# 다시 말해 결정 노드 하나와 리프 노드 두 개로 이루어진 트리입니다. 이 트리가 AdaBoostClassifier의 기본 추정기입니다.
# AdaBoostRegressor는 깊이가 3인 DecisionTreeRegressor를 기본 추정기로 사용합니다.

from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
DecisionTreeClassifier(max_depth=1), n_estimators=200,
algorithm='SAMME.R', learning_rate=0.5)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(class_weight=None,
                                                         criterion='gini',
                                                         max_depth=1,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort=False,
                                                         random_state=None,
                             

## 그래디언트 부스팅(Gradient Boosting)(p258)

아다부스트처럼 그래디언트 부스팅은 앙상블에 이전까지의 오차를 보정하도록 예측기를 순차적으로 추가합니다. 하지만 아다부스트처럼 반복마다 샘플의 가중치를 수정하는 대신 이전 예측기가 만든 잔여 오차(residual error)에 새로운 예측기를 학습시킵니다.

In [29]:
# 결정 트리를 기반 예측기로 사용하는 간단한 회귀 문제를 풀어보겠습니다.
# 당연히 그래디언트 부스팅은 회귀 문제에도 아주 잘 작동합니다.
# 이를 그래디언트 트리 부스팅(Gradient Tree Boosting) 또는 그래디언트 부스티드 회귀 트리(Gradient Boosted Regression Tree, GBRT)라고 합니다.
# 먼저 DecisionTreeRegressor를 훈련 세트(잡음이 섞인 2차 곡선 형태의 훈련 세트)에 학습시켜보겠습니다.

import numpy as np
from sklearn.tree import DecisionTreeRegressor

np.random.seed(42)
X = np.random.rand(100, 1) - 0.5
y = 3*X[:, 0]**2 + 0.05 * np.random.randn(100)

tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X,y)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [30]:
# 이제 첫 번째 예측기에서 생긴 잔여 오차에 두 번째 DecisionTreeRegressor를 훈련시킵니다.

y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X, y2)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [31]:
# 그 다음 두 번째 예측기가 만든 잔여 오차에 세 번째 회귀 모델을 훈련시킵니다.

y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X, y3)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [32]:
# 이제 세 개의 트리를 포함하는 앙상블 모델이 생겼습니다.
# 새로운 샘플에 대한 예측을 만들려면 모든 트리의 예측을 더하면 됩니다.

X_new = np.array([[0.8]])
y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))
print(y_pred)

[0.75026781]


![image.png](https://i.imgur.com/ywfpnUK.png)

위 그림의 왼쪽 열은 이 세 트리의 예측이고 오른쪽 열은 앙상블의 예측입니다. 첫 번째 행에서는 앙상블에 트리가 하나만 있어서 첫 번째 트리의 예측과 완전히 같습니다. 두 번째 행에서는 새로운 트리가 첫 번째 트리의 잔여 오차에 대해 학습되었습니다. 오른쪽의 앙상블 예측이 두 개의 트리 예측의 합과 같은 것을 볼 수 있습니다. 비슷하게 세 번째 행에서는 또 다른 트리가 두 번째 트리의 잔여 오차에 훈련되었습니다. 트리가 앙상블에 추가될수록 앙상블의 예측이 점차 좋아지는 것을 알 수 있습니다.

In [33]:
# 사이킷런의 GradientBoostingRegressor를 사용하면 GBRT 앙상블을 간단하게 훈련시킬 수 있습니다.
# 트리 수(n_estimators)와 같이 앙상블의 훈련을 제어하는 매개변수는 물론 RandomForestRegressor와 아주 비슷하게 결정 트리의 성장을 제어하는 매개변수(max_depth, min_samples_leaf 등)를 가지고 있습니다.
# 다음은 이전에 만든 것과 같은 앙상블을 만드는 코드입니다.

from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth = 2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X, y)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=1.0, loss='ls', max_depth=2,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=3,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

learning_rate 매개변수가 각 트리의 기여 정도를 조절합니다. 이를 0.1처럼 낮게 설정하면 앙상블을 훈련 세트에 학습시키기 위해 많은 트리가 필요하지만 일반적으로 예측의 성능은 좋아집니다. 이는 축소(shrinkage)라고 부르는 규제 방법입니다. 아래 그림은 작은 학습률로 훈련시킨 두 개의 GBRT 앙상블을 보여줍니다. 왼쪽은 훈련 세트를 학습하기에는 트리가 충분하지 않은 반면 오른쪽은 트리가 너무 많아 훈련 세트에 과대적합되었습니다.

![image.png](https://i.imgur.com/WD1hh1g.png)

최적의 트리 수를 찾기 위해서는 조기 종료 기법을 사용할 수 있습니다. 간단하게 구현하려면 staged_predict() 메서드를 사용합니다. 이 메서드는 훈련의 각 단계(트리 하나, 트리 두 개 등)에서 앙상블에 의해 만들어진 예측기를 순회하는 반복자(iterator)를 반환합니다.

In [50]:
# 다음 코드는 120개의 트리로 GBRT 앙상블을 훈련시키고 최적의 트리 수를 찾기 위해 각 훈련 단계에서 검증오차를 측정합니다.
# 마지막에 최적의 트리 수를 사용해 새로운 GBRT 앙상블을 훈련시킵니다.

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X, y)

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train, y_train)

errors = [mean_squared_error(y_val, y_pred)
         for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors)

print('best n_estimators :',bst_n_estimators)
gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators)
gbrt_best.fit(X_train, y_train)

best n_estimators : 52


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=2,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=52,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

![image.png](https://i.imgur.com/pKIsX15.png)

위 그림의 왼쪽은 검증 오차고 오른쪽은 최적 모델의 예측입니다. (많은 수의 트리를 먼저 훈련시키고 최적의 수를 찾기 위해 살펴보는 대신) 실제로 훈련을 중지하는 방법으로 조기 종료를 구현할 수도 있습니다. warm_start=True로 설정하면 사이킷런이 fit() 메서드가 호출될 때 기존 트리를 유지하고 훈련을 추가할 수 있도록 해줍니다.

In [51]:
# 다음 코드는 연속해서 다섯 번의 반복 동안 검증 오차가 향상되지 않으면 훈련을 멈춥니다.

gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)

min_val_error = float('inf')
error_going_up = 0
for n_estimators in range(1,120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train, y_train)
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred)
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up = 0
    else:
        error_going_up += 1
        if error_going_up == 5:
            break # 조기 종료

GradientBoostingRegressor는 각 트리가 훈련할 때 사용할 훈련 샘플의 비율을 지정할 수 있는 subsample 매개변수도 지원합니다. 예를 들어 subsample=0.25라고 하면 각 트리는 무작위로 선택된 25%의 훈련 샘플로 학습됩니다. 아마 추측할 수 있겠지만 편향이 높아지는 대신 분산이 낮아지게 됩니다. 또한 훈련 속도를 상당히 높입니다. 이런 기법을 확률적 그래디언트 부스팅(Stochastic Gradient Boosting)이라고 합니다. 그래디언트 부스팅에 다른 비용 함수를 사용할 수도 있습니다. loss 매개변수를 이용해 지정합니다.

## 스태킹(stacking, stacked generalization)(p263)

* 이는 '앙상블에 속한 모든 예측기의 예측을 취합하는 간단한 함수(직접 투표 같은)를 사용하는 대신 취합하는 모델을 훈련시킬 수 없을까요?'라는 기본 아이디어로 출발합니다.<br>
![image.png](https://i.imgur.com/uWK2yX7.png)<br>
* 위 그림은 새로운 샘플에 회귀 작업을 수행하는 앙상블을 보여주고 있습니다. 아래의 세 예측기는 각각 다른 값(3.1, 2.7, 2.9)을 예측하고 마지막 예측기(블렌더(blender)또는 메타 학습기(meta learner)라고 합니다)가 이 예측을 입력으로 받아 최종 예측(3.0)을 만듭니다.

![image.png](https://i.imgur.com/SJ9Nd85.png)

블렌더를 학습시키는 일반적인 방법은 홀드 아웃(hold-out)세트를 사용하는 것입니다. 어떻게 작동하는지 살펴보겠습니다. 먼저 훈련 세트를 두 개의 서브셋으로 나눕니다. 첫 번째 서브셋은 첫 번째 레이어의 예측을 훈련시키기 위해 사용됩니다. 그런 다음 첫 번째 레이어의 예측기를 사용해 두 번째 (홀드 아웃) 세트에 대한 예측을 만듭니다. 예측기들이 훈련하는 동안 이 샘플들을 전혀 보지 못했기 때문에 이때 만들어진 예측은 완전히 새로운 것입니다. 이제 홀드 아웃 세트의 각 샘플에 대해 세 개의 예측값이 만들어집니다. 타깃값은 그대로 쓰고 앞에서 예측한 값을 입력 특성으로 사용하는 새로운 훈련 세트를 만들 수 있습니다(새로운 훈련 세트는 3차원이 됩니다). 블렌더가 새 훈련 세트로 훈련됩니다. 즉, 첫 번째의 레이어의 예측을 가지고 타깃값을 예측하도록 학습됩니다.

사실 이런 방식의 블렌더를 여러 개 훈련시키는 것도 가능합니다(예를 들어 하나는 선형 회귀로, 다른 하나는 랜덤 포레스트 회귀로, 등등). 그러면 블렌더만의 레이어가 만들어지게 됩니다. 이렇게 하려면 훈련 세트를 세 개의 서브셋으로 나눕니다. 첫 번째 세트는 첫 번째 레이어를 훈련시키는 데 사용되고 두 번째 세트는 (첫 번째 레이어의 예측기로) 두 번째 레이러를 훈련시키기 위한 훈련 세트를 만드는 데 사용됩니다. 그리고 세 번째 세트는 (두 번째 레이어의 예측기로) 세 번째 레이어를 훈련시키기 위한 훈련 세트를 만드는 데 사용됩니다. 작업이 끝나면 아래 그림처럼 각 레이어를 차례대로 실행해서 새로운 샘플에 대한 예측을 만들 수 있습니다.

![image.png](https://i.imgur.com/grJsMSr.png)

### 연습문제

#### 1. 정확히 같은 훈련 데이터로 다섯 개의 다른 모델을 훈련시켜서 모두 95%의 정확도를 얻었다면 이 모델들을 연결하여 더 좋은 결과를 얻을 수 있을까요? 가능하다면 어떻게 해야 할까요? 그렇지 않다면 왜일까요?
다섯 개의 모델을 훈련시켰고 모두 95%의 정확도를 달성했다면 이들을 연결하여 투표 앙상블(voting ensemble)을 만들어 더 나은 결과를 기대할 수 있습니다. 만약 모델이 서로 다르다면(예를 들면 SVM 분류기, 결정 트리 분류기, 로지스틱 회귀 분류기 등) 훨씬 좋습니다. 만약 다른 훈련 샘플에서 훈련되었다면 더더욱 좋습니다(이것이 배깅과 페이스팅 앙상블의 핵심입니다). 하지만 그렇지 않더라도 모델이 서로 많이 다르면 여전히 좋은 결과를 얻어냅니다.
<br><br>

#### 2. 직접 투표와 간접 투표 분류기 사이의 차이점은 무엇인가요?
직접 투표 분류기는 앙상블에 있는 각 분류기의 선택을 카운트해서 가장 많은 투표를 얻은 클래스를 선택합니다. 간접 투표 분류기는 각 클래스의 평균적인 확률 추정값을 계산해서 가장 높은 확률을 가진 클래스를 고릅니다. 이 방식은 신뢰가 높은 투표에 더 가중치를 주고 종종 더 나은 성능을 냅니다. 하지만 앙상블에 있는 모든 분류기가 클래스 확률을 추정할 수 있어야 사용할 수 있습니다(예를 들어 사이킷런의 SVM 분류기는 probability=True로 지정해야 합니다).
<br><br>

#### 3. 배깅 앙상블의 훈련을 여러 대의 서버에 분산시켜 속도를 높일 수 있을까요? 페이스팅 앙상블, 부스팅 앙상블, 랜덤 포레스트, 스태킹 앙상블의 경우는 어떨까요?
배깅 앙상블의 각 예측기는 독립적이므로 여러 대의 서버에 분산하여 앙상블의 훈련 속도를 높일 수 있습니다. 페이스팅 앙상블과 랜덤 포레스트도 같은 이유로 동일합니다. 그러나 부스팅 앙상블의 예측기는 이전 예측기를 기반으로 만들어지므로 훈련이 순차적이어야하고 여러 대의 서버에 분산해서 얻을 수 있는 이득이 없습니다. 스태킹 앙상블의 경우 한 층의 모든 예측기가 각각 독립적이므로 여러 대의 서버에서 병렬로 훈련될 수 있습니다. 그러나 한 층에 있는 예측기들은 이전 층의 예측기들이 훈련된 후에 훈련될 수 있습니다.
<br><br>

#### 4. oob 평가의 장점은 무엇인가요?
oob 평가를 사용하면 배깅 앙상블의 각 예측기가 훈련에 포함되지 않은(즉, 따로 떼어놓은) 샘플을 사용해 평가됩니다. 이는 추가적인 검증 세트가 없어도 편향되지 않게 앙상블을 평가하도록 도와줍니다. 그러므로 훈련에 더 많은 샘플을 사용할 수 있어서 앙상블의 성능은 조금 더 향상될 것입니다.
<br><br>

#### 5. 무엇이 엑스트라 트리를 일반 랜덤 포레스트보다 더 무작위하게 만드나요? 추가적인 무작위성이 어떻게 도움이 될까요? 엑스트라 트리는 일반 랜덤 포레스트보다 느릴까요, 빠를까요?
랜덤 포레스트에서 트리가 성장할 때 각 노드에서 특성의 일부를 무작위로 선택해 분할에 사용합니다. 엑스트라 트리에서도 이는 마찬가지지만 한 단계 더 나아가서 일반 결정 트리처럼 가능한 최선의 임계점을 찾는 것이 아니라 각 특성에 대해 랜덤한 임계점을 사용합니다. 이 추가적인 무작위성은 규제처럼 작동합니다. 즉, 랜덤 포레스트가 훈련 데이터에 과대적합되었다면 엑스트라 트리는 그렇지 않을 것입니다. 또한 엑스트라 트리는 가능한 최선의 임계점을 찾지 않기 때문에 랜덤 포레스트보다 훨씬 빠르게 훈련됩니다. 그러나 예측을 할 때는 랜덤 포레스트보다 더 빠르지도 느리지도 않습니다.
<br><br>

#### 6. 아다부스트 앙상블이 훈련 데이터에 과소적합되었다면 어떤 매개변수를 어떻게 바꾸어야 할까요?
아다부스트 앙상블이 훈련 데이터에 과소적합되었다면 예측기 수를 증가시키거나 기반 예측기의 규제 하이퍼파라미터를 감소시켜 볼 수 있습니다. 또한 학습률을 약간 증가시켜 볼 수 있습니다.
<br><br>

#### 7. 그래디언트 부스팅 앙상블이 훈련 데이터에 과대적합되었다면 학습률을 높여야 할까요, 낮춰야 할까요?
그래디언트 부스팅 앙상블이 훈련 세트에 과대적합되었다면 학습률을 감소시켜야 합니다. (예측기 수가 너무 많으면) 알맞은 개수를 찾기 위해 조기 종료 기법을 사용할 수 있습니다.